In [1]:
%cd ../../
%pwd

c:\Users\vince\Documents\school\Unief-6\6-bap


'c:\\Users\\vince\\Documents\\school\\Unief-6\\6-bap'

In [2]:
from chatbot_domain.benchmark import JsonQuestionParser, Benchmarker, AlwaysATestSubject, NLPTestSubject, HumanTestSubject
from chatbot_domain.chatbot import ChatBotModifier, OpenAIChatBot, DIPDomainGuard, ChatBotBuilder, RAGBuilder
from chatbot_domain.rag import VectorRetriever, FacebookDPR
from chatbot_domain.data import loadDataSetFromDisk
from chatbot_domain import logger

logger.setLevel("INFO")

# chatbot = ChatBotBuilder.openAI('gpt-4-turbo').benchmarkGuard().domainGuard(DIPDomainGuard).rag(
#     RAGBuilder.fromDatasetDisk('ragData.set').vectorRetriever(FacebookDPR()), contextLength = 1024 * 8
#     ).build()

parser = JsonQuestionParser('questions.json')
questions = parser.parse()

name = input("name: ")
benchmarker = Benchmarker(HumanTestSubject(name), questions)
testResult = benchmarker.evaluate()
print(testResult)
testResult.save('./scores')

TypeError: HumanTestSubject.__init__() missing 1 required positional argument: 'name'